# Train a Linear Regression Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

In this notebook, you will learn how to use the Watson Machine Learning Accelerator (WML-A) API and accelerate the processing of Linear Regression model on GPU with Watson Machine Learning Accelerator.

Linear Regression is a simple machine learning model where the response y is modelled by a linear combination of the predictors in X.

In this notebook we have three versions of Linear Regression model: scikit-learn version, cuML version and snapML version.

All three versions will be submitted onto WMLA. And we can compare the performance benifit of cuML and snapML version.

This notebook covers the following sections:

1. [Setup Linear Regression using sklearning](#skl-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#skl-cpu)<br>

1. [Setup Linear Regression using cuML](#cuml-model)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#cuml-gpu)<br>

1. [Setup Linear Regression using snapML](#snapml-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#snap-cpu)<br>

1. [Setup Linear Regression using snapML GPU](#snapml-model-gpu)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#snap-gpu)<br>

## Preparations
### Prepare directory and file for writing Linear Regression engine.

In [1]:
from pathlib import Path
model_dir = f'/data/models' 
model_base_name = f'linearRegression-main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


<a id = "skl-model"></a>
## Step 1 : Setup Linear Regression model using scikit-learn.
### Create a Linear Regression Model based on scikit-learn on CPU

In [2]:
model_main='sklean-'+model_base_name

In [3]:
%%writefile {model_dir}/{model_main}

import os, datetime
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
import numpy as np

# Define Parameters for a large regression
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399
random_state = 23

# Generate Data
start = datetime.datetime.now()
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)
X_skl, X_skl_test, y_skl, y_skl_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
end = datetime.datetime.now()
print ("generate data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_skl.shape[0]))
print("Number of features: %d" % (X_skl.shape[1]))

# scikit-learn Model
lr_model = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_skl, y_skl)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict_skl = lr_model.predict(X_skl_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score_skl = r2_score(y_skl_test, predict_skl)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

#  print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (scikit-learn): %.4f" % r2_score_skl)

Overwriting /data/models/sklean-linearRegression-main.py


<a id = "skl-cpu"></a>
## Step 2 :  Training the SK-Learning model on CPU with Watson Machine Learning Accelerator
### Prepare the model lib for job submission

In [4]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [5]:
# please enter Watson Machine Learning Accelerator host name
hostname='wmla-console-wmla.apps.dse-perf.cpolab.ibm.com'
# login='username:password' # please enter the login and password
login='mluser1:mluser1'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)

a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)

dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)

https://wmla-console-wmla.apps.dse-perf.cpolab.ibm.com/auth/v1/logon


### Define the status checking function

In [6]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

### Define the submission function

In [7]:
def submit_job_to_wmla (args) :
    starttime = datetime.datetime.now()
    r = requests.post(dl_rest_url+'/execs?args='+args, # files=files,
                  headers=commonHeaders, verify=False)
    if not r.ok:
        print('submit job failed: code=%s, %s'%(r.status_code, r.content))
    job_status = query_job_status(r.json(),refresh_rate=5)
    endtime = datetime.datetime.now()
    print("\nTotallly training cost: ", (endtime - starttime).seconds, " seconds.")

<a id = "cpu"></a>
### Define the submission parameters for scikit-learn version on cpu

In [8]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType cpu \
--conda-env-name dlipy3-cpu  \
--model-main /gpfs/mydatafs/models/' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --conda-env-name dlipy3-cpu  --model-main /gpfs/mydatafs/models/sklean-linearRegression-main.py


### Submit WMLA Workload

In [42]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-599,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --...,wmla-599,mluser1,FINISHED,wmla-599,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-599/_submitted_code,SingleNodeTensorflowTrain,2021-07-20T08:18:58Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-599',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu '
          '--conda-env-name dlipy3-cpu  --model-main '
          '/gpfs/mydatafs/models/sklean-linearRegression-main.py ',
  'createTime': '2021-07-20T08:18:58Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-599',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-599',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-599/_submitted_code'}

Totallly training cost:  277  seconds.


<a id = "cuml-model"></a>
## Step 3 :  Setup Linear Regression model using cmML
### Create a Linear Regression Model based on cuML on GPU

In [9]:
model_main='cuML-'+model_base_name

In [10]:
%%writefile {model_dir}/{model_main}

import os, datetime
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
import numpy as np

# specify the cache location to /gpfy since ~/.cache is not available
os.environ["CUPY_CACHE_DIR"]="/gpfs/mydatafs/models/cache/lr"

# Define Parameters for a large regression
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399
random_state = 23

# Generate Data
start = datetime.datetime.now()
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)
X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]
X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
end = datetime.datetime.now()
print ("generate data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_cudf.shape[0]))
print("Number of features: %d" % (X_cudf.shape[1]))

# cuML Model
lr_model = cuLinearRegression(fit_intercept=True,normalize=True,algorithm='eig')

# Fit
start = datetime.datetime.now()
lr_model.fit(X_cudf, y_cudf)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict_cuml = lr_model.predict(X_cudf_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

print("R^2 score (cuML): %.4f" % r2_score_cuml)

Overwriting /data/models/cuML-linearRegression-main.py


<a id = "cuml-gpu"></a>
## Step 4 :  Training the cuML model on GPU with Watson Machine Learning Accelerator
### Re-define the submssion parameters

In [11]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType gpu \
--conda-env-name rapids-21.06-new \
--model-main /gpfs/mydatafs/models/' + model_main

print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --conda-env-name rapids-21.06-new --model-main /gpfs/mydatafs/models/cuML-linearRegression-main.py


### Submit WMLA Workload

In [12]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-626,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --...,wmla-626,mluser1,FINISHED,wmla-626,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-626/_submitted_code,SingleNodeTensorflowTrain,2021-07-20T09:59:31Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-626',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu '
          '--conda-env-name rapids-21.06-new --model-main '
          '/gpfs/mydatafs/models/cuML-linearRegression-main.py ',
  'createTime': '2021-07-20T09:59:31Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-626',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-626',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-626/_submitted_code'}

Totallly training cost:  51  seconds.


<a id = "snaml-model"></a>
## Step 5 :  Setup Linear Regression model using snapML
### Create a Linear Regression Model based on snapML 

In [47]:
model_main='snapML-'+model_base_name

In [48]:
%%writefile {model_dir}/{model_main}

import datetime
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from snapml import LinearRegression as SnapLinearRegression
import numpy as np

# Define Parameters for a large regression
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399
random_state = 23

# Generate Data
start = datetime.datetime.now()
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
end = datetime.datetime.now()
print ("generate data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# snapML model
lr_model = SnapLinearRegression(fit_intercept=True,normalize=True)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

r2_score_snapml = r2_score(y_test, lr_model.predict(X_test))
print("R^2 score (snapML): %.4f" % r2_score_snapml)

Writing /data/models/snapML-linearRegression-main.py


<a id = "snapml-cpu"></a>
## Step 6 :  Training the SnapML model on CPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [49]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType cpu \
--conda-env-name snapml-177rc \
--model-main /gpfs/mydatafs/models/' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --conda-env-name snapml-177rc --model-main /gpfs/mydatafs/models/snapML-linearRegression-main.py


### Submit WMLA Workload

In [50]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-601,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu --...,wmla-601,mluser1,FINISHED,wmla-601,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-601/_submitted_code,SingleNodeTensorflowTrain,2021-07-20T08:23:59Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-601',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType cpu '
          '--conda-env-name snapml-177rc --model-main '
          '/gpfs/mydatafs/models/snapML-linearRegression-main.py ',
  'createTime': '2021-07-20T08:23:59Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-601',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-601',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-601/_submitted_code'}

Totallly training cost:  255  seconds.


<a id = "snaml-model-gpu"></a>
## Step 7 :  Setup Linear Regression model using snapML on GPU
### Create a Linear Regression Model based on snapML 

In [51]:
model_main='snapML-gpu-'+model_base_name

In [52]:
%%writefile {model_dir}/{model_main}

import datetime
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from snapml import LinearRegression as SnapLinearRegression
import numpy as np

# Define Parameters for a large regression
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399
random_state = 23

# Generate Data
start = datetime.datetime.now()
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
end = datetime.datetime.now()
print ("generate data timecost: %.2gs" % ((end-start).total_seconds()))

print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))

# snapML model
lr_model = SnapLinearRegression(use_gpu=True,
                                fit_intercept=True,normalize=True)

# Fit
start = datetime.datetime.now()
lr_model.fit(X_train, y_train)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

r2_score_snapml = r2_score(y_test, lr_model.predict(X_test))
print("R^2 score (snapML): %.4f" % r2_score_snapml)

Writing /data/models/snapML-gpu-linearRegression-main.py


<a id = "snapml-gpu"></a>
## Step 8 :  Training the SnapML model on GPU with Watson Machine Learning Accelerator
### Re-define the submission parameters

In [53]:
# specify the model file, conda env, device type and device number
args = '--exec-start tensorflow --cs-datastore-meta type=fs \
--workerDeviceNum 1 \
--workerMemory 32G \
--workerDeviceType gpu \
--conda-env-name snapml-177rc \
--model-main /gpfs/mydatafs/models/' + model_main
print(args)

--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --conda-env-name snapml-177rc --model-main /gpfs/mydatafs/models/snapML-gpu-linearRegression-main.py


In [54]:
submit_job_to_wmla (args)

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-602,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu --...,wmla-602,mluser1,FINISHED,wmla-602,https://wmla-mss:9080,wmla,/gpfs/myresultfs/mluser1/batchworkdir/wmla-602/_submitted_code,SingleNodeTensorflowTrain,2021-07-20T08:28:14Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-602',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta type=fs '
          '--workerDeviceNum 1 --workerMemory 32G --workerDeviceType gpu '
          '--conda-env-name snapml-177rc --model-main '
          '/gpfs/mydatafs/models/snapML-gpu-linearRegression-main.py ',
  'createTime': '2021-07-20T08:28:14Z',
  'creator': 'mluser1',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-602',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-602',
  'workDir': '/gpfs/myresultfs/mluser1/batchworkdir/wmla-602/_submitted_code'}

Totallly training cost:  82  seconds.
